# Prepare the input data objects


```micromamba activate gena-env```

In [1]:
import numpy as np
import pandas as pd
import multianndata as mad
import cna
import scanpy as sc
import matplotlib.pyplot as plt
np.random.seed(0)

In [2]:
# constants
outdir = (
    "/directflow/SCCGGroupShare/projects/blabow/tenk10k_phase1/data_processing/csa_qtl"
)

1. prepare the required inputs:
    * cell metadata with donor info for each cell 
    * cell by gene counts matrix 
    * donor-level metadata storing covariate information to be used in the GWAS, as well as batches

In [3]:
# read in the latest tenk cohort
adata = sc.read(
    "/directflow/SCCGGroupShare/projects/blabow/tenk10k_phase1/data_processing/scanpy/output/integrated_objects/240_libraries/240_libraries_concatenated_harmony_leiden_filtered_reanalysed.h5ad",
    backed='r'
)
# NOTE: reading in backed mode is 100x faster although I can't modify the object 
# NOTE: can also try out "caching" the h5ad for faster read times as well??

## Currently we have only TOB genotypes available on brenner
# filter for just TOB to do a practice run
adata = adata[adata.obs["cohort"] == "TOB"]
### ^^ Remove this when I want to do analysis for Bioheart

In [4]:
with pd.option_context('display.max_columns', None):
    display(adata.obs.head(3))

,cellbender_background_fraction,cellbender_cell_probability,cellbender_cell_size,cellbender_droplet_efficiency,celltypist_predicted_labels,celltypist_over_clustering,celltypist_majority_voting,celltypist_conf_score,wg2_sample,wg2_nCount_RNA,wg2_nFeature_RNA,wg2_percent_mt,wg2_azimuth_predicted_celltype_l2,wg2_azimuth_predicted_celltype_l2_score,wg2_scpred_prediction,Vireo_Individual_Assignment,Vireo_DropletType,scDblFinder_DropletType,scDblFinder_Score,scds_score,scds_DropletType,MajoritySinglet_DropletType,MajoritySinglet_Individual_Assignment,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,total_counts_hb,pct_counts_hb,original_barcode,new_cell_name,sequencing_library,individual,cohort,onek1k_id,cpg_id_old,tob_id,cpg_id,onek1k_donor,ct_id,batch,leiden
AAACCCAAGAATCGAT_S0012b,0.009700,0.999954,7748.034668,1.187165,Tcm/Naive helper T cells,4,Tcm/Naive helper T cells,0.988154,S0012b,8454.0,2621.0,2.247457,CD4_TCM,0.681302,CD4_Naive,558,singlet,singlet,0.007869,0.279380,singlet,singlet,558,2621,2621,8454.0,190.0,2.247457,2676.0,31.653654,2.0,0.023657,AAACCCAAGAATCGAT-1,AAACCCAAGAATCGAT_S0012b,S0012b,CPG313965,TOB,557_558,CPG6106,TOB1413,CPG313965,558,NaN,0,6
AAACCCAAGAGCAACC_S0012b,0.018032,0.999963,7818.318848,0.806677,Tem/Temra cytotoxic T cells,3,Tem/Temra cytotoxic T cells,0.339064,S0012b,5823.0,2328.0,5.667182,CD4_CTL,0.616923,CD4_CTL,27,singlet,doublet,0.782162,0.102471,singlet,singlet,27,2328,2328,5823.0,330.0,5.667181,957.0,16.434828,0.0,0.000000,AAACCCAAGAGCAACC-1,AAACCCAAGAGCAACC_S0012b,S0012b,CPG309211,TOB,27_27,CPG943,TOB1642,CPG309211,27,NaN,0,2
AAACCCAAGCGACAGT_S0012b,0.262001,0.999995,4126.960449,0.783668,Tem/Trm cytotoxic T cells,49,Tem/Temra cytotoxic T cells,0.011127,S0012b,2729.0,1519.0,14.254306,CD8_TEM,0.461040,CD8_TEM,502,singlet,singlet,0.000019,0.094898,singlet,singlet,502,1519,1519,2729.0,389.0,14.254306,84.0,3.078051,1.0,0.036643,AAACCCAAGCGACAGT-1,AAACCCAAGCGACAGT_S0012b,S0012b,CPG313437,TOB,501_502,CPG5546,TOB1337,CPG313437,502,NaN,0,2


Note: looks like there are 42 TOB samples with CPG ids without onek1k study id's. Possible that they have no WGS and therefore no CPG ids, or they are somehow missing for the master metadata sheet. I will just remove these for now...

In [5]:
cell_meta = adata.obs[["onek1k_id"]].set_index(adata.obs.index).rename(columns={"onek1k_id": "id"})
cell_meta.to_csv(f"{outdir}/data/cell_meta.csv")
display(cell_meta.head(3))

,id
AAACCCAAGAATCGAT_S0012b,557_558
AAACCCAAGAGCAACC_S0012b,27_27
AAACCCAAGCGACAGT_S0012b,501_502


In [6]:
# get seq library to cpg id mapping - use this to remove batchy neighbourhoods later on
sequencing_library_mapping = adata.obs[
    ["onek1k_id", "sequencing_library"]
].drop_duplicates()
sequencing_library_mapping = sequencing_library_mapping.set_index(
    "onek1k_id", drop=True
)
sequencing_library_mapping["batch"] = sequencing_library_mapping[
    "sequencing_library"
].factorize()[0]
display(sequencing_library_mapping.head(3))
batch_mapping = sequencing_library_mapping.drop("sequencing_library", axis=1)
display(batch_mapping)
sequencing_library_mapping.to_csv(f"{outdir}/data/sequencing_library_mapping.csv")

,sequencing_library,batch
onek1k_id,,
557_558,S0012b,0
27_27,S0012b,0
501_502,S0012b,0


,batch
onek1k_id,
557_558,0
27_27,0
501_502,0
491_492,0
500_501,0
...,...
391_392,119
348_349,119
330_331,119


 NOTE:
 * Figure out how to add "batch" to samplem and remove batchy neighbourhoods when running cna 
 * currently this is not possible / can't figure out how to do it when I have a many:many mapping between batches and samples (onek1k id's)?
 * this just makes the samplem have multiple rows per sample which feels wrong 
 * can maybe find an answer to this in 
 

In [23]:
with pd.option_context("display.max_rows", None):
    display(sequencing_library_mapping.sort_index().head(100))

,sequencing_library,batch
onek1k_id,,
1_1,S0026c,56
1_1,S0026a,61
1_1,S0026b,48
1_1,S0025_28b,59
1_1,S0025_28a,52
2_2,S0026b,48
2_2,S0026a,61
2_2,S0026c,56
2_2,S0025_28a,52


In [7]:
# get the counts matrix
cells_x_genes = adata.to_df()
# export counts to csv
display(cells_x_genes.head(3))
# cells_x_genes.to_csv(f"{outdir}/data/cells_x_genes.csv")

,ENSG00000187961,ENSG00000187583,ENSG00000188290,ENSG00000131591,ENSG00000186891,ENSG00000186827,ENSG00000162572,ENSG00000149527,ENSG00000162591,ENSG00000238260,...,ENSG00000183878,ENSG00000290853,ENSG00000165246,ENSG00000176728,ENSG00000291031,ENSG00000291033,ENSG00000229236,ENSG00000198692,ENSG00000289707,ENSG00000228253
AAACCCAAGAATCGAT_S0012b,-0.098161,-0.063251,-0.087368,-0.272086,-0.240088,-0.262634,-0.068546,-0.150766,-0.213132,-0.073337,...,-0.763635,-0.202512,-0.143501,-0.242200,-0.185859,-0.508295,-0.194552,-0.412900,-0.261451,-1.648947
AAACCCAAGAGCAACC_S0012b,-0.103190,-0.068927,-0.052226,-0.253554,-0.236903,-0.237149,-0.069453,-0.168430,-0.207583,-0.075447,...,-0.772065,-0.184258,-0.150796,-0.208313,-0.185392,-0.486838,-0.178072,-0.334451,-0.223021,0.583485
AAACCCAAGCGACAGT_S0012b,-0.115717,-0.093197,-0.121048,-0.250837,-0.193814,-0.167083,-0.073713,-0.195072,-0.187337,-0.075682,...,-0.834996,-0.212822,-0.170287,-0.190746,-0.175803,-0.457842,-0.173275,-0.198170,-0.120289,0.288104


In [8]:
sample_meta = pd.read_csv(
    "/directflow/SCCGGroupShare/projects/anncuo/TenK10K_pilot/tenk10k/saige_qtl_input_files_covariates_sex_age_geno_pcs_tob_bioheart.csv"
)
sample_meta = sample_meta[sample_meta["sample_id"].isin(adata.obs["cpg_id"])]
# add in the onek1k id
cpg_onek1k_id_mapping = adata.obs[["onek1k_id", "cpg_id"]].drop_duplicates()
display(cpg_onek1k_id_mapping.head(3))
sample_meta = pd.merge(
    sample_meta,
    cpg_onek1k_id_mapping,
    how="left",
    left_on="sample_id",
    right_on="cpg_id",
)
sample_meta = sample_meta.set_index("onek1k_id", drop=True).drop(
    ["sample_id", "cpg_id"], axis=1
) # remove these columns as I don't think the multianndata supports string variables in samplem 
display(sample_meta.head(3))

,onek1k_id,cpg_id
AAACCCAAGAATCGAT_S0012b,557_558,CPG313965
AAACCCAAGAGCAACC_S0012b,27_27,CPG309211
AAACCCAAGCGACAGT_S0012b,501_502,CPG313437


,sex,age,geno_PC1,geno_PC2,geno_PC3,geno_PC4,geno_PC5,geno_PC6,geno_PC7,geno_PC8,geno_PC9,geno_PC10,geno_PC11,geno_PC12,geno_PC13,geno_PC14,geno_PC15,geno_PC16
onek1k_id,,,,,,,,,,,,,,,,,,
276_277,2.0,65.0,0.031874,-0.000410,0.004952,-0.000557,0.001924,-0.022447,0.002643,-0.001893,0.008205,-0.004277,0.003711,-0.006016,0.011691,-0.009226,0.001659,0.007041
278_279,1.0,45.0,0.031191,0.004006,0.003113,-0.002887,0.003075,-0.024638,0.002322,0.002359,0.021236,-0.005583,0.004636,-0.006784,0.014104,-0.007854,-0.003376,0.004050
279_280,2.0,73.0,0.032146,0.001124,0.002034,-0.003210,-0.000880,-0.015663,0.003080,-0.001562,0.014510,-0.003109,0.004357,-0.001489,0.008275,-0.007595,-0.005692,0.002729


In [9]:
madata = mad.MultiAnnData(X=cells_x_genes, obs=cell_meta, sampleid="id")
# Add all covariate information to d.samplem
madata.samplem = madata.samplem.join(sample_meta).join(batch_mapping)
# check that you can actually save it with current metadata columns
madata.write(f"{outdir}/data/scDataObject.h5ad")

['id']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


In [10]:
madata

AnnData object with n_obs × n_vars = 2798264 × 3770
    obs: 'id'
    uns: 'sampleXmeta'

In [11]:
madata.samplem

,sex,age,geno_PC1,geno_PC2,geno_PC3,geno_PC4,geno_PC5,geno_PC6,geno_PC7,geno_PC8,geno_PC9,geno_PC10,geno_PC11,geno_PC12,geno_PC13,geno_PC14,geno_PC15,geno_PC16,batch
id,,,,,,,,,,,,,,,,,,,
1_1,2.0,65.0,0.028473,0.004421,0.002113,-0.001891,0.002678,-0.012493,0.002679,-0.003106,0.010715,-0.002058,0.000756,-0.000088,0.007350,-0.008075,0.001969,0.003025,48
1_1,2.0,65.0,0.028473,0.004421,0.002113,-0.001891,0.002678,-0.012493,0.002679,-0.003106,0.010715,-0.002058,0.000756,-0.000088,0.007350,-0.008075,0.001969,0.003025,52
1_1,2.0,65.0,0.028473,0.004421,0.002113,-0.001891,0.002678,-0.012493,0.002679,-0.003106,0.010715,-0.002058,0.000756,-0.000088,0.007350,-0.008075,0.001969,0.003025,56
1_1,2.0,65.0,0.028473,0.004421,0.002113,-0.001891,0.002678,-0.012493,0.002679,-0.003106,0.010715,-0.002058,0.000756,-0.000088,0.007350,-0.008075,0.001969,0.003025,59
1_1,2.0,65.0,0.028473,0.004421,0.002113,-0.001891,0.002678,-0.012493,0.002679,-0.003106,0.010715,-0.002058,0.000756,-0.000088,0.007350,-0.008075,0.001969,0.003025,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081_1082,1.0,73.0,0.024420,0.002767,0.001645,-0.000627,0.003350,-0.010090,0.001389,-0.000284,0.015945,-0.006433,0.001119,-0.001997,0.006709,-0.009851,-0.001108,0.008631,38
1081_1082,1.0,73.0,0.024420,0.002767,0.001645,-0.000627,0.003350,-0.010090,0.001389,-0.000284,0.015945,-0.006433,0.001119,-0.001997,0.006709,-0.009851,-0.001108,0.008631,40
1081_1082,1.0,73.0,0.024420,0.002767,0.001645,-0.000627,0.003350,-0.010090,0.001389,-0.000284,0.015945,-0.006433,0.001119,-0.001997,0.006709,-0.009851,-0.001108,0.008631,57


In [12]:
sample_meta

,sex,age,geno_PC1,geno_PC2,geno_PC3,geno_PC4,geno_PC5,geno_PC6,geno_PC7,geno_PC8,geno_PC9,geno_PC10,geno_PC11,geno_PC12,geno_PC13,geno_PC14,geno_PC15,geno_PC16
onek1k_id,,,,,,,,,,,,,,,,,,
276_277,2.0,65.0,0.031874,-0.000410,0.004952,-0.000557,0.001924,-0.022447,0.002643,-0.001893,0.008205,-0.004277,0.003711,-0.006016,0.011691,-0.009226,0.001659,0.007041
278_279,1.0,45.0,0.031191,0.004006,0.003113,-0.002887,0.003075,-0.024638,0.002322,0.002359,0.021236,-0.005583,0.004636,-0.006784,0.014104,-0.007854,-0.003376,0.004050
279_280,2.0,73.0,0.032146,0.001124,0.002034,-0.003210,-0.000880,-0.015663,0.003080,-0.001562,0.014510,-0.003109,0.004357,-0.001489,0.008275,-0.007595,-0.005692,0.002729
280_281,2.0,25.0,0.022925,-0.001849,0.008604,-0.003138,0.001227,-0.012471,-0.000189,-0.000706,0.019296,-0.005162,0.004675,-0.011633,0.001585,-0.012458,0.001691,0.002326
281_282,2.0,53.0,0.029691,0.004519,0.004947,-0.002352,0.002671,-0.021663,0.005412,-0.002743,0.019496,-0.011135,0.008282,-0.007981,0.013814,-0.013639,0.000209,0.009120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959_960,2.0,19.0,0.030631,0.003893,0.004568,-0.002958,0.001628,-0.020637,0.003194,-0.002507,0.025744,-0.007174,0.001096,-0.013423,0.013252,-0.015006,-0.001747,0.010526
960_961,2.0,73.0,0.029611,0.001636,0.002385,-0.001334,0.000352,-0.015766,0.002925,0.001546,0.023328,-0.002993,0.004423,-0.005617,0.009979,-0.014310,0.001826,0.009812
961_962,2.0,62.0,0.027068,0.004368,0.005220,-0.003066,0.002257,-0.016874,0.000897,-0.002162,0.018960,-0.006821,0.003939,-0.007990,0.009144,-0.013954,-0.006080,0.001970


# Data vis

In [ ]:
# Visualize data
# plt.scatter(madata.obsm['X_umap'][d.obs.clust1==1,0], d.obsm['X_umap'][d.obs.clust1==1,1],
#             c="green", edgecolor='none', s=2, label = "Cluster 1")
# plt.scatter(madata.obsm['X_umap'][d.obs.clust2==1,0], d.obsm['X_umap'][d.obs.clust2==1,1],
#             c="purple", edgecolor='none', s=2, label = "Cluster 2")
# plt.scatter(madata.obsm['X_umap'][d.obs.clust3==1,0], d.obsm['X_umap'][d.obs.clust3==1,1],
#             c="orange", edgecolor='none', s=2, label = "Cluster 3")
# plt.legend(loc="lower left", markerscale=7, frameon=False)
# plt.axis("off")
# plt.show()

### Other required inputs:
* plink2 format genotyping data for each sample